<a href="https://colab.research.google.com/github/venanciojamilly/desafio-care-intuitive/blob/main/Experimento_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [ ]:
!uv pip install langchain langchain-community langchain-openai langchain-huggingface pypdf chromadb faiss-cpu

Using Python 3.12.11 environment at: /usr
Resolved 117 packages in 7.79s
Prepared 28 packages in 4.28s
Uninstalled 1 package in 2ms
Installed 28 packages in 91ms
 + backoff==2.2.1
 + bcrypt==4.3.0
 + chromadb==1.1.0
 + coloredlogs==15.0.1
 + dataclasses-json==0.6.7
 + durationpy==0.10
 + faiss-cpu==1.12.0
 + httptools==0.6.4
 + humanfriendly==10.0
 + kubernetes==33.1.0
 + langchain-community==0.3.29
 + langchain-huggingface==0.3.1
 + langchain-openai==0.3.33
 + marshmallow==3.26.1
 + mmh3==5.2.0
 + mypy-extensions==1.1.0
 + onnxruntime==1.22.1
 + opentelemetry-exporter-otlp-proto-common==1.37.0
 + opentelemetry-exporter-otlp-proto-grpc==1.37.0
 + opentelemetry-proto==1.37.0
 + posthog==5.4.0
 + pybase64==1.4.2
 + pypdf==6.1.0
 + pypika==0.48.9
 - requests==2.32.4
 + requests==2.32.5
 + typing-inspect==0.9.0
 + uvloop==0.21.0
 + watchfiles==1.1.0


In [ ]:
# Para lidar com os dados em PDF
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Para indexação em bases vetoriais (salvar e extrair os dados coletados)
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma

# Para consultas com APIs fornecedoras de LLMs para inferência (chat models e prompts)
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate
)

# Chain para fazer o RAG de fato
from langchain.chains import RetrievalQA

# Agente + Tools
from langchain.agents import initialize_agent, Tool, AgentType

In [ ]:
import re, requests, hashlib

from pathlib import Path
from typing import Iterable, List

# Baixar PDFs

In [ ]:
def safe_filename_from_url(url: str) -> str:
    tail = url.split("/")[-1].split("?")[0] or "file.pdf"
    if not tail.lower().endswith(".pdf"):
        tail += ".pdf"
    tail = re.sub(r"[^A-Za-z0-9._-]+", "_", tail)
    h = hashlib.md5(url.encode("utf-8")).hexdigest()[:8]
    return f"{Path(tail).stem}_{h}.pdf"

def baixar_pdfs(urls: Iterable[str], dest_dir: str = "data/pdfs", timeout: int = 60) -> List[str]:
  Path(dest_dir).mkdir(parents=True, exist_ok=True)
  salvos = []
  for url in urls:
    fname = safe_filename_from_url(url)
    fpath = Path(dest_dir) / fname
    r = requests.get(url, stream=True, timeout=timeout, verify=False)
    r.raise_for_status()
    ctype = (r.headers.get("content-type") or "").lower()
    if "pdf" not in ctype and not url.lower().endswith(".pdf"):
        print(f"O 'content-type' não indica que é PDF ({ctype}). Mas será salvo mesmo assim.")
    with open(fpath, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    salvos.append(str(fpath))
    print(f"Baixado: {url} -> {fpath}")
  return salvos

In [ ]:
urls = [
    "https://www.ufcg.edu.br/~costa/resolucoes/res_16082017.pdf", # Resolução PPC de computação
    "https://www.prac.ufcg.edu.br/images/editais/2025/Edital_CGAE_N_4_2025_1.pdf", # Edital Auxílio
]

In [ ]:
pdf_paths = baixar_pdfs(urls)

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ufcg.edu.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Baixado: https://www.ufcg.edu.br/~costa/resolucoes/res_16082017.pdf -> data/pdfs/res_16082017_5b3b4f03.pdf


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.prac.ufcg.edu.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Baixado: https://www.prac.ufcg.edu.br/images/editais/2025/Edital_CGAE_N_4_2025_1.pdf -> data/pdfs/Edital_CGAE_N_4_2025_1_1673fadb.pdf


# Chunks

In [ ]:
def load_and_split_pdfs(filepaths: Iterable[str], chunk_size: int = 800, chunk_overlap: int = 120) -> list:
  docs = []

  for fp in filepaths:
    loader = PyPDFLoader(fp)
    docs.extend(loader.load())

  splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap,
      add_start_index=True,
      separators=["\n\n", "\n", " ", ""],
  )
  chunks = splitter.split_documents(docs)
  print(f"Documentos: {len(docs)} | Chunks: {len(chunks)}")
  return chunks

In [ ]:
#chunks = load_and_split_pdfs(pdf_paths, chunk_size=900, chunk_overlap=150)
chunks = load_and_split_pdfs(pdf_paths, chunk_size=200, chunk_overlap=40)

Documentos: 28 | Chunks: 263


In [ ]:
import pprint

pprint.pprint(chunks)

[Document(metadata={'producer': '', 'creator': 'Doro PDF Writer [1.88] [http://j.mp/the_sz]', 'creationdate': '2017-11-06T11:18:47-03:00', 'moddate': '2017-11-06T11:18:47-03:00', 'title': 'RESOLUÇÃO Nº 08.2017 - PPC de Computação - CEEI', 'subject': 'RESOLUÇÃO Nº 08.2017 - PPC de Computação - CEEI', 'author': 'SODS', 'keywords': 'RESOLUÇÃO Nº 08.2017 - PPC de Computação - CEEI', 'source': 'data/pdfs/res_16082017_5b3b4f03.pdf', 'total_pages': 14, 'page': 0, 'page_label': '1', 'start_index': 0}, page_content='SERVIÇO PÚBLICO FEDERAL \nUNIVERSIDADE FEDERAL DE CAMPINA GRANDE \nCONSELHO UNIVERSITÁRIO \nCÂMARA SUPERIOR DE ENSINO \n \nRESOLUÇÃO Nº 08/2017  \n \nAltera a estrutura curricular do Curso de Ciência d a'),
 Document(metadata={'producer': '', 'creator': 'Doro PDF Writer [1.88] [http://j.mp/the_sz]', 'creationdate': '2017-11-06T11:18:47-03:00', 'moddate': '2017-11-06T11:18:47-03:00', 'title': 'RESOLUÇÃO Nº 08.2017 - PPC de Computação - CEEI', 'subject': 'RESOLUÇÃO Nº 08.2017 - PPC de

In [ ]:
docs = []

for fp in pdf_paths:
  loader = PyPDFLoader(fp)
  docs.extend(loader.load())

print(docs)

[Document(metadata={'producer': '', 'creator': 'Doro PDF Writer [1.88] [http://j.mp/the_sz]', 'creationdate': '2017-11-06T11:18:47-03:00', 'moddate': '2017-11-06T11:18:47-03:00', 'title': 'RESOLUÇÃO Nº 08.2017 - PPC de Computação - CEEI', 'subject': 'RESOLUÇÃO Nº 08.2017 - PPC de Computação - CEEI', 'author': 'SODS', 'keywords': 'RESOLUÇÃO Nº 08.2017 - PPC de Computação - CEEI', 'source': 'data/pdfs/res_16082017_5b3b4f03.pdf', 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content='SERVIÇO PÚBLICO FEDERAL \nUNIVERSIDADE FEDERAL DE CAMPINA GRANDE \nCONSELHO UNIVERSITÁRIO \nCÂMARA SUPERIOR DE ENSINO \n \nRESOLUÇÃO Nº 08/2017  \n \nAltera a estrutura curricular do Curso de Ciência d a \nComputação, modalidade Bacharelado, do Centro de \nEngenharia Elétrica e Informática, Campus de Campin a \nGrande, fixada pela Resolução CONSEPE/UFPB nº\n \n58/99 e dá outras providências. \n \nA Câmara Superior de Ensino do Conselho Universitár io da Universidade Federal de Campina \nGrande, no us

# Modelo de Embeddings

In [ ]:
from google.colab import files
uploaded = files.upload() # É preciso passar as chaves da API desejada (OpenAI e/ou HuggingFace)

Saving .env to .env


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [ ]:
# Modelo de Embeddings da OpenAI (pago)
embeddings_oa = OpenAIEmbeddings(model="text-embedding-3-small")

# Modelo de Embeddings do HuggingFace (free)
name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings_hf = HuggingFaceEmbeddings(
    model_name=name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Base vetorial

## FAISS

In [ ]:
# FAISS
faiss_index = FAISS.from_documents(chunks, embeddings_oa)

In [ ]:
retriever_faiss = faiss_index.as_retriever(search_type="mmr", search_kwargs={"k": 5, "fetch_k": 20})
res_faiss = retriever_faiss.invoke("o que fala o Art. 7º")
#res_faiss = retriever_faiss.invoke("Qual o componente curricular do 9º período")
#res_faiss = retriever_faiss.invoke("quais os CONTEÚDOS OBRIGATÓRIOS")
#res_faiss = retriever_faiss.invoke("quais os componentes curriculares do 4 período")

print(f"Retornados: {len(res_faiss)} documentos")
print("\n\n---\n\n".join([doc.page_content for doc in res_faiss]))

Retornados: 5 documentos
Art. 17.  Esta Resolução entra em vigência na data de sua pu blicação, revogadas as 
disposições em contrário.

---

7.2.1  Para interpor recurso, o(a) candidato/a deve protocolar processo por meio 
do SEI -UFCG, optando pelo tipo de processo “Assistência Estudantil:

---

período letivo. 
 
Art. 7º
 A estrutura curricular do Curso, constituída por C omponentes Curriculares 
Obrigatórios, Componentes Curriculares Optativos de  Formação Específica e Componentes

---

APÊNDICE- I

---

7.  DA HOMOLOGAÇÃO DAS INSCRIÇÕES 
 
7.1 Após as inscrições, a PRAC/CGAE irá submetê -las à avaliação dos critérios 
dispostos no item 3 deste Edital.


## Chroma

In [ ]:
# Chroma
chroma_index = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_oa,
    collection_name="ufcg_pdfs",
    persist_directory="data/chroma_index"
)
chroma_index.persist()

/tmp/ipython-input-1376346488.py:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_index.persist()


In [ ]:
# Carregar vectorstore
chroma_index = Chroma(
    collection_name="ufcg_pdfs",
    embedding_function=embeddings_oa,
    persist_directory="./data/chroma_index"
)

print("Número de embeddings salvos: ", chroma_index._collection.count())

retriever_chroma = chroma_index.as_retriever(search_type="mmr", search_kwargs={"k": 5, "fetch_k": 20})
#res_chroma = retriever_chroma.invoke("quantas vagas tem para pessoas com deficiência para os programas de auxílio?")
#res_chroma = retriever_chroma.invoke("quais são os tipos de componentes do curso de ciência da computação?")
res_chroma = retriever_chroma.invoke("sobre o que fala o Art. 7º")
print(f"Retornados: {len(res_chroma)} documentos")
print("\n\n---\n\n".join([doc.page_content for doc in res_chroma]))

Número de embeddings salvos:  263
Retornados: 5 documentos
Art. 17.  Esta Resolução entra em vigência na data de sua pu blicação, revogadas as 
disposições em contrário.

---

7.  DA HOMOLOGAÇÃO DAS INSCRIÇÕES 
 
7.1 Após as inscrições, a PRAC/CGAE irá submetê -las à avaliação dos critérios 
dispostos no item 3 deste Edital.

---

APÊNDICE- I

---

período letivo. 
 
Art. 7º
 A estrutura curricular do Curso, constituída por C omponentes Curriculares 
Obrigatórios, Componentes Curriculares Optativos de  Formação Específica e Componentes

---

7.2.1  Para interpor recurso, o(a) candidato/a deve protocolar processo por meio 
do SEI -UFCG, optando pelo tipo de processo “Assistência Estudantil:


# Retrieval

## BM25

In [ ]:
!uv pip install rank_bm25

Using Python 3.12.11 environment at: /usr
Resolved 2 packages in 107ms
Prepared 1 package in 16ms
Installed 1 package in 1ms
 + rank-bm25==0.2.2


In [ ]:
from langchain.retrievers import BM25Retriever

In [ ]:
bm25_retriever = BM25Retriever.from_documents(docs)

In [ ]:
result = bm25_retriever.invoke("o que fala o Art. 4º")
#result = bm25_retriever.invoke("quais os componentes curriculares do 4 período")

print(f"Retornados: {len(result)} documentos")
print("\n\n---\n\n".join([doc.page_content for doc in result]))

print(result)

Retornados: 4 documentos
R E S O L V E : 
Art. 1º Aprovar a estrutura curricular fixada no Projeto Pedagógico do Curso de Ciência 
da Computação, na modalidade Bacharelado, do Centro  de Engenharia Elétrica e Informática, 
Campus de Campina Grande, desta Universidade.  
 
Art. 2º
 O Curso de Graduação em Ciência da Computação tem como finalidade conferir 
o grau de Bacharel aos alunos que cumprirem as dete rminações constantes na presente 
Resolução e demais normas da Instituição. 
 
Art. 3º
 Atribuir-se-á a cada componente curricular um total de créditos, de modo que 01 
(um) crédito corresponderá a 15 (quinze) horas. 
 
Art. 4º
 O Curso terá a duração mínima de 3.270 (três mil, duzentas e setenta) horas, 
correspondendo a 218 (duzentos e dezoito) créditos,  conforme o demonstrativo no quadro a 
seguir: 
 
Tipo de Componente  Horas  Créditos  % 
Obrigatório  1980 132 60,55% 
Optativo Geral  240 16 7,34% 
Optativo Específico  600 40 18,35% 
Trabalho de Conclusão de Curso  120 8 3,67% 


## Retriever Ensemble

In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

en_retriever = EnsembleRetriever(
    retrievers=[retriever_faiss, bm25_retriever],
    weights=[0.7, 0.3]
)

In [ ]:
res = en_retriever.invoke("o que fala o Art 4º")
#res = retriever.invoke("quais os componentes curriculares do quarto período")
#res = retriever.invoke("quantas vagas tem para auxílio transporte?")

print(f"Retornados: {len(res)} documentos")
print("\n\n---\n\n".join([doc.page_content for doc in res]))

Retornados: 9 documentos
Art. 17.  Esta Resolução entra em vigência na data de sua pu blicação, revogadas as 
disposições em contrário.

---

Art. 4º
 O Curso terá a duração mínima de 3.270 (três mil, duzentas e setenta) horas, 
correspondendo a 218 (duzentos e dezoito) créditos,  conforme o demonstrativo no quadro a 
seguir:

---

A
LARCON AGRA DO Ó 
Presidente

---

apresentar, no ato do ingresso, a guarda legal da criança, a carteira de vacinação e 
comprovante de matrícula escolar (para crianças a partir de 4 anos). 
 
13.  DO PAGAMENTO DOS AUXÍLIOS

---

APÊNDICE- I

---

R E S O L V E : 
Art. 1º Aprovar a estrutura curricular fixada no Projeto Pedagógico do Curso de Ciência 
da Computação, na modalidade Bacharelado, do Centro  de Engenharia Elétrica e Informática, 
Campus de Campina Grande, desta Universidade.  
 
Art. 2º
 O Curso de Graduação em Ciência da Computação tem como finalidade conferir 
o grau de Bacharel aos alunos que cumprirem as dete rminações constantes na present

# RetrieverQA e Tool

## RetrieverQA

In [ ]:
llm = ChatOpenAI(model_name="gpt-4o-2024-08-06", temperature=0)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=en_retriever,
    return_source_documents=True
)

In [ ]:
query = "Sobre o que fala o Art. 4?"
res = qa_chain.invoke(query)

print("Resposta:\n", res["result"])
print("\n--- Fontes ---")
for doc in res["source_documents"]:
    print(doc.metadata, "\n", doc.page_content, "\n")

Resposta:
 O Art. 4º diz respeito ao Curso de Graduação em Ciência da Computação, especificamente estabelecendo que o curso terá a duração mínima de 3.270 horas, correspondendo a 218 créditos, com a carga horária distribuída entre componentes obrigatórios, optativos gerais, optativos específicos, trabalho de conclusão de curso e atividades complementares flexíveis.

--- Fontes ---
{'producer': '', 'creator': 'Doro PDF Writer [1.88] [http://j.mp/the_sz]', 'creationdate': '2017-11-06T11:18:47-03:00', 'moddate': '2017-11-06T11:18:47-03:00', 'title': 'RESOLUÇÃO Nº 08.2017 - PPC de Computação - CEEI', 'subject': 'RESOLUÇÃO Nº 08.2017 - PPC de Computação - CEEI', 'author': 'SODS', 'keywords': 'RESOLUÇÃO Nº 08.2017 - PPC de Computação - CEEI', 'source': 'data/pdfs/res_16082017_5b3b4f03.pdf', 'total_pages': 14, 'page': 3, 'page_label': '4', 'start_index': 1096} 
 Art. 17.  Esta Resolução entra em vigência na data de sua pu blicação, revogadas as 
disposições em contrário. 

{'producer': '', 'c

## Tools

Ferramenta (tool) de exemplo para fazer scraping em páginas web (neste caso, portal de últimas notícias da prac da UFCG) e utilizar no agente.

In [ ]:
import requests
from langchain_core.tools import tool
from bs4 import BeautifulSoup

@tool
def buscar_noticias_ufcg(query: str) -> str:
    """
    Ler o conteúdo do portal de notícias da UFCG.
    """
    try:
        url = "https://www.prac.ufcg.edu.br/ultimas-noticias"
        print(f"O agente está visitando o site: {url}")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        for script in soup(["script", "style"]):
            script.decompose()

        text = soup.get_text(separator='\n', strip=True)

        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)

        return text[:50000]

    except requests.RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"Error processing the webpage: {str(e)}"

In [ ]:
print(buscar_noticias_ufcg.invoke(""))

O agente está visitando o site: https://www.prac.ufcg.edu.br/ultimas-noticias
Últimas Notícias
Ir direto para menu de acessibilidade.
Seu navegador de internet está sem suporte à JavaScript. Por esse motivo algumas funcionalidades do site podem não estar acessíveis.
Portal do Governo Brasileiro
Atualize sua Barra de Governo
Ir para o conteúdo
1
Ir para o menu
2
Ir para a busca
3
Ir para o rodapé
4
Opções de acessibilidade
Acessibilidade
Alto contraste
Mapa do site
Pró-Reitoria de
Assuntos Comunitários
Universidade Federal de Campina Grande
Buscar no portal
Busca
Buscar no portal
Busca:
Buscar
Redes Sociais
Facebook
Instagram
Twitter
Serviços
Equipe
Estrutura
Notícias
Contato
Fim do menu de serviços
Página inicial
>
Últimas Notícias
Menu
Início do menu principal
Menu de Relevância
Editais
Termos e Documentos
Programas
Residência Universitária
Restaurante Universitário
Programa Bolsa Permanência
Auxílios
Auxílio ao Ensino de Graduação
Monitoria Inclusiva
Restaurante Universitário
Auxílio

Adicionar o RetrievalQA criado como tool + a tool de scraping de sites:

In [ ]:
tools = [
    Tool(
        name="Consultar documentos da UFCG.",
        func=lambda q: qa_chain({"query": q})["result"],
        description="Recupera documentos oficiais da UFCG a partir de uma base de dados."
    ),
    buscar_noticias_ufcg
]

pprint.pprint(tools)

[Tool(name='Consultar documentos da UFCG.', description='Recupera documentos oficiais da UFCG a partir de uma base de dados.', func=<function <lambda> at 0x79faba638d60>),
 StructuredTool(name='buscar_noticias_ufcg', description='Ler o conteúdo do portal de notícias da UFCG.', args_schema=<class 'langchain_core.utils.pydantic.buscar_noticias_ufcg'>, func=<function buscar_noticias_ufcg at 0x79faba638220>)]


# Agente

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    """
    Você é um assistente especializado em consultas acadêmicas da UFCG.
    Sua função é ajudar estudantes e servidores a obter informações de forma clara e fundamentada.

    Para isso, você pode utilizar duas ferramentas:
    1. 'Consultar documentos da UFCG.' -> Use quando a pergunta for sobre resoluções, editais, regulamentos ou documentos oficiais.
    2. 'read_page' -> Use somente para buscar informações atualizadas em notícias, sempre use exatamente este link para acessar o site: `https://www.prac.ufcg.edu.br/ultimas-noticias`.

    Se a pergunta for fora desses contextos, explique educadamente que só pode responder sobre documentos e notícias oficiais da UFCG.
    """
)

## Inicializar o Agente

In [ ]:
agente = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={
        "system_message": system_prompt
    },
    verbose=True
)

## Chat

Um mini chat para testar o agente em execução

In [ ]:
def chatbot_agentic_rag(agente):
    print("Inicializando o mini chatbot com agente/rag! Digite 'sair' para encerrar a sessão.")
    while True:
        user_query = input("Você: ")
        if user_query.lower() == "sair":
            print("Sessão encerrada.")
            break
        try:
            response = agente.invoke(user_query)
            print(f"Bot: {response}")
        except Exception as e:
            print(f"Error: {e}")

In [ ]:
chatbot_agentic_rag(agente)

Inicializando o mini chatbot com agente/rag! Digite 'sair' para encerrar a sessão.
Você: quais as últimas notícias da ufcg?


> Entering new AgentExecutor chain...
Action: buscar_noticias_ufcg
Action Input: "últimas notícias"O agente está visitando o site: https://www.prac.ufcg.edu.br/ultimas-noticias

Observation: Últimas Notícias
Ir direto para menu de acessibilidade.
Seu navegador de internet está sem suporte à JavaScript. Por esse motivo algumas funcionalidades do site podem não estar acessíveis.
Portal do Governo Brasileiro
Atualize sua Barra de Governo
Ir para o conteúdo
1
Ir para o menu
2
Ir para a busca
3
Ir para o rodapé
4
Opções de acessibilidade
Acessibilidade
Alto contraste
Mapa do site
Pró-Reitoria de
Assuntos Comunitários
Universidade Federal de Campina Grande
Buscar no portal
Busca
Buscar no portal
Busca:
Buscar
Redes Sociais
Facebook
Instagram
Twitter
Serviços
Equipe
Estrutura
Notícias
Contato
Fim do menu de serviços
Página inicial
>
Últimas Notícias
Menu
Início do menu